In [1]:
%matplotlib inline
import json
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import os
# import seaborn as sns
from datetime import datetime
import re

In [2]:
folder = "../Data"
files = ["round1-2.json", "allrounds.json", "round1.json", "zumbi-venom-11-07-session2.json"]
# files = ["round1.json"]
sessions = {}
for file in files:
    path = os.path.join(folder, file)
    raw_data = pd.json_normalize(pd.read_json(path)['execution_stream'])
    session_key = file.split('.')[0]
    sessions[session_key] = raw_data

In [3]:
for key in sessions:
    print(f"{key} -\n first_message: {sessions[key].head(1)['topic']} -\n last_message: {sessions[key].tail(1)['topic']}\n ------")

round1-2 -
 first_message: 0    2019-05-18T13:35:43.454Z-7160cfbc-e6f1-d8e5-5c...
Name: topic, dtype: object -
 last_message: 33263    2019-05-18T13:34:25.765Z-d6d76852-943c-0ee4-8e...
Name: topic, dtype: object
 ------
allrounds -
 first_message: 0    2019-05-18T20:06:23.809Z-6988c9c5-b742-72b1-83...
Name: topic, dtype: object -
 last_message: 102645    2019-05-18T13:34:25.765Z-d6d76852-943c-0ee4-8e...
Name: topic, dtype: object
 ------
round1 -
 first_message: 0    2019-05-18T13:35:43.454Z-7160cfbc-e6f1-d8e5-5c...
Name: topic, dtype: object -
 last_message: 19903    2019-05-18T13:34:25.765Z-d6d76852-943c-0ee4-8e...
Name: topic, dtype: object
 ------
zumbi-venom-11-07-session2 -
 first_message: 0    2019-07-11T14:20:52.342Z-a5b70eec-48b3-2044-55...
Name: topic, dtype: object -
 last_message: 10185    data/case/148e8e25-5f91-4189-83cd-646c1ebed299...
Name: topic, dtype: object
 ------


## Os arquivos de backup mais antigos são de 18-05-2019 apenas o *zumbi-venom-11-07-session2* inclui o férias mas essa sessão também se encontra no *executions-bkp-30-08-19.json*

In [4]:
bkp_filename = "executions-bkp-30-08-19.jsonl"
path = os.path.join(folder, bkp_filename)
bkp_data = pd.read_json(path, lines=True)
bkp_data.head(5)

,_id,topic,payload,timestamp
0,{'$oid': '5d274101febb1ac74b9ce573'},data/case/148e8e25-5f91-4189-83cd-646c1ebed299...,{'responseStamp': 1},2019-07-11 14:00:33.384
1,{'$oid': '5d274101febb1ac74b9ce574'},data/theme/zombie.case/get,{'responseStamp': 2},2019-07-11 14:00:33.470
2,{'$oid': '5d274101febb1ac74b9ce575'},data/case/148e8e25-5f91-4189-83cd-646c1ebed299...,"{'name': 'Zombie Venom 01', 'source': '# 1 (ca...",2019-07-11 14:00:33.475
3,{'$oid': '5d274101febb1ac74b9ce576'},data/theme/zombie.case/get/response/2,"{'content': '<div class=""styt-pul-main"">{knot}...",2019-07-11 14:00:33.786
4,{'$oid': '5d274101febb1ac74b9ce577'},data/theme/zombie.start/get,{'responseStamp': 3},2019-07-11 14:00:33.789


In [5]:
# Filtrar time stamp para 11/07/2019 das 11 as 14 GMT-03:00
data_session_11_07 = bkp_data.loc[lambda df: df.timestamp > '2019-07-11 14:00:00']
data_session_11_07 = data_session_11_07.loc[lambda df: df.timestamp < '2019-07-11 17:00:00']
data_session_11_07
# data_venom.to_json("executions-experiments-zv-group1.json", orient='split')

,_id,topic,payload,timestamp
0,{'$oid': '5d274101febb1ac74b9ce573'},data/case/148e8e25-5f91-4189-83cd-646c1ebed299...,{'responseStamp': 1},2019-07-11 14:00:33.384
1,{'$oid': '5d274101febb1ac74b9ce574'},data/theme/zombie.case/get,{'responseStamp': 2},2019-07-11 14:00:33.470
2,{'$oid': '5d274101febb1ac74b9ce575'},data/case/148e8e25-5f91-4189-83cd-646c1ebed299...,"{'name': 'Zombie Venom 01', 'source': '# 1 (ca...",2019-07-11 14:00:33.475
3,{'$oid': '5d274101febb1ac74b9ce576'},data/theme/zombie.case/get/response/2,"{'content': '<div class=""styt-pul-main"">{knot}...",2019-07-11 14:00:33.786
4,{'$oid': '5d274101febb1ac74b9ce577'},data/theme/zombie.start/get,{'responseStamp': 3},2019-07-11 14:00:33.789
...,...,...,...,...
17623,{'$oid': '5d276592febb1ac74b9d2f5e'},control/knot/Início/selected,{'content': ''},2019-07-11 16:36:34.239
17624,{'$oid': '5d276592febb1ac74b9d2f5f'},data/theme/minimal.picture_up_left/get,{'responseStamp': 13},2019-07-11 16:36:34.244
17625,{'$oid': '5d276595febb1ac74b9d2f60'},data/theme/minimal.picture_up_left/get/respons...,"{'content': '<div class=""styt-pul-main"">{knot}...",2019-07-11 16:36:37.056
17626,{'$oid': '5d276595febb1ac74b9d2f61'},data/template/Cancel/get,{'responseStamp': 14},2019-07-11 16:36:37.059


In [6]:
# Filtrar time stamp para 15/07/2019 das 10 as 14 GMT-03:00
data_session_15_07 = bkp_data.loc[lambda df: df.timestamp > '2019-07-15 12:00:00']
data_session_15_07 = data_session_15_07.loc[lambda df: df.timestamp < '2019-07-15 17:00:00']
data_session_15_07

,_id,topic,payload,timestamp
18439,{'$oid': '5d2c6efb739d8a1fc75aa8c1'},data/user/login,"{'email': 'jacinto@example.com', 'password': '...",2019-07-15 12:18:03.515
18440,{'$oid': '5d2c6efb739d8a1fc75aa8c2'},data/user/login/response/1,"{'userid': 1, 'token': 'eyJhbGciOiJIUzI1NiIsIn...",2019-07-15 12:18:03.859
18441,{'$oid': '5d2c6efb739d8a1fc75aa8c3'},data/case/*/list,"{'filterBy': 'user', 'filter': 1, 'responseSta...",2019-07-15 12:18:03.874
18442,{'$oid': '5d2c6eff739d8a1fc75aa8c4'},data/case/*/list/response/2,[{'id': '024c72c2-7251-4ff5-b848-65585fc362ee'...,2019-07-15 12:18:07.314
18443,{'$oid': '5d2c6f07739d8a1fc75aa8c5'},data/case/024c72c2-7251-4ff5-b848-65585fc362ee...,{'responseStamp': 3},2019-07-15 12:18:15.782
...,...,...,...,...
31541,{'$oid': '5d2c9276739d8a1fc75ae094'},2019-07-15T14:43:26.353Z-79cbf93b-70d1-cbcc-af...,"{'track': {'userid': 23, 'caseid': '09e94ada-6...",2019-07-15 14:49:26.348
31542,{'$oid': '5d2c9276739d8a1fc75ae095'},2019-07-15T14:43:26.353Z-79cbf93b-70d1-cbcc-af...,"{'responseStamp': 33, 'track': {'userid': 23, ...",2019-07-15 14:49:26.352
31543,{'$oid': '5d2c9276739d8a1fc75ae096'},2019-07-15T14:43:26.353Z-79cbf93b-70d1-cbcc-af...,"{'content': 'Previous Knot', 'track': {'userid...",2019-07-15 14:49:26.357
31544,{'$oid': '5d2c9276739d8a1fc75ae097'},2019-07-15T14:43:26.353Z-79cbf93b-70d1-cbcc-af...,"{'content': '<div class=""styt-pul-main"">{knot}...",2019-07-15 14:49:26.361


## Reconstructing user track

In [7]:
def reconstruct_users_track(log):
    users_tracks = dict()
    for index, row in log.iterrows():
        payload = row.payload
        if 'track' in payload and 'userid' in payload['track']:
            user = payload['track']['userid'] 
            if user in users_tracks:
                users_tracks[user].append(row)
            else:
                users_tracks.update({user: [row]})
    return users_tracks

In [8]:
def users_points_history(users_track, get_raw=False):
    users_points = dict()
    for user in users_track.keys():
        for log in users_track[user]:
            if 'var/points/set' in log['topic']:
#               print(log['payload'])
                points=log['payload']['content']
                if get_raw:
                    case_id = log['payload']['track']['caseid']
                    if user in users_points:
                        users_points[user].append((points, case_id))
                    else:
                        users_points.update({user: [(points, case_id)]})
                else:
                    if user in users_points:
                        users_points[user].append(points)
                    else:
                        users_points.update({user: [points]})
    return users_points

In [9]:
def dic_to_dataFrames(dictionary):
    data_frames = []
    for key in dictionary.keys():
        df = pd.DataFrame(dictionary[key],columns=[key])
        data_frames.append(df)

    return pd.concat(data_frames, axis=1)

In [10]:
def count_hit_and_miss(users_track):
    hits_misses = dict()
    for user in users_track.keys():
        hits   = 0
        misses = 0
        for log in users_track[user]:
            if  re.search('knot/\w+_certo/start', log['topic']):
                hits += 1
            if  re.search('knot/\w+_errado/start', log['topic']):
                misses += 1
        hits_misses[user] = {'hits':hits, 'misses':misses}
    return hits_misses

## Processamento de grupos

In [11]:
session_11_07_track = reconstruct_users_track(data_session_11_07)

In [12]:
session_11_07_track.keys()

dict_keys([21, 1, 17, 19, 16, 23, 22, 20, 18, 27, 31, 26, 32, 28, 30, 29, 24])

In [13]:
session_15_07_track = reconstruct_users_track(data_session_15_07)

In [14]:
session_15_07_track.keys()

dict_keys([1, 7, 9, 21, 10, 12, 11, 19, 14, 18, 16, 17, 26, 25, 23, 24])

In [151]:
user_points_session_11_07 = users_points_history(session_11_07_track)
for k in user_points_session_11_07:
    if user_points_session_11_07[k][0] != 100:
        user_points_session_11_07[k].insert(0, 100)

In [152]:
# Dividindo rodadas do usuário 17 e 26
user_points_session_11_07[15] = user_points_session_11_07[17][17:]
user_points_session_11_07[25] = user_points_session_11_07[26][17:]

In [153]:
user_points_session_11_07 = dic_to_dataFrames(user_points_session_11_07)
user_points_session_11_07 

,21,1,17,19,16,23,22,20,18,27,31,26,32,28,30,29,24,15,25
0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,90.0,100.0,90.0,90.0,110.0,90.0,90.0,90.0,90.0,110.0,90.0,110,90.0,90.0,110.0,110.0,90.0,110.0,90.0
2,80.0,NaN,80.0,80.0,100.0,80.0,100.0,80.0,80.0,100.0,100.0,100,100.0,100.0,100.0,120.0,100.0,100.0,100.0
3,90.0,NaN,90.0,90.0,90.0,70.0,110.0,70.0,70.0,110.0,90.0,90,90.0,90.0,110.0,110.0,90.0,110.0,90.0
4,80.0,NaN,100.0,80.0,100.0,60.0,100.0,80.0,60.0,100.0,80.0,100,100.0,80.0,100.0,100.0,100.0,120.0,80.0
5,90.0,NaN,90.0,70.0,110.0,70.0,110.0,90.0,50.0,90.0,70.0,110,90.0,90.0,110.0,110.0,90.0,110.0,90.0
6,80.0,NaN,100.0,80.0,100.0,80.0,120.0,80.0,60.0,80.0,80.0,100,80.0,100.0,100.0,100.0,80.0,100.0,100.0
7,90.0,NaN,90.0,90.0,90.0,90.0,110.0,90.0,70.0,70.0,70.0,90,70.0,NaN,90.0,110.0,90.0,90.0,110.0
8,100.0,NaN,80.0,100.0,100.0,80.0,120.0,80.0,80.0,60.0,60.0,80,60.0,NaN,100.0,120.0,100.0,80.0,120.0
9,110.0,NaN,70.0,110.0,110.0,70.0,110.0,90.0,90.0,NaN,70.0,70,50.0,NaN,90.0,110.0,110.0,NaN,110.0


In [154]:
user_points_session_11_07 = user_points_session_11_07[:][:17]
user_points_session_11_07

,21,1,17,19,16,23,22,20,18,27,31,26,32,28,30,29,24,15,25
0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,90.0,100.0,90.0,90.0,110.0,90.0,90.0,90.0,90.0,110.0,90.0,110,90.0,90.0,110.0,110.0,90.0,110.0,90.0
2,80.0,NaN,80.0,80.0,100.0,80.0,100.0,80.0,80.0,100.0,100.0,100,100.0,100.0,100.0,120.0,100.0,100.0,100.0
3,90.0,NaN,90.0,90.0,90.0,70.0,110.0,70.0,70.0,110.0,90.0,90,90.0,90.0,110.0,110.0,90.0,110.0,90.0
4,80.0,NaN,100.0,80.0,100.0,60.0,100.0,80.0,60.0,100.0,80.0,100,100.0,80.0,100.0,100.0,100.0,120.0,80.0
5,90.0,NaN,90.0,70.0,110.0,70.0,110.0,90.0,50.0,90.0,70.0,110,90.0,90.0,110.0,110.0,90.0,110.0,90.0
6,80.0,NaN,100.0,80.0,100.0,80.0,120.0,80.0,60.0,80.0,80.0,100,80.0,100.0,100.0,100.0,80.0,100.0,100.0
7,90.0,NaN,90.0,90.0,90.0,90.0,110.0,90.0,70.0,70.0,70.0,90,70.0,NaN,90.0,110.0,90.0,90.0,110.0
8,100.0,NaN,80.0,100.0,100.0,80.0,120.0,80.0,80.0,60.0,60.0,80,60.0,NaN,100.0,120.0,100.0,80.0,120.0
9,110.0,NaN,70.0,110.0,110.0,70.0,110.0,90.0,90.0,NaN,70.0,70,50.0,NaN,90.0,110.0,110.0,NaN,110.0


In [155]:
# Removendo usuário de teste
del user_points_session_11_07[1]
user_points_session_11_07

,21,17,19,16,23,22,20,18,27,31,26,32,28,30,29,24,15,25
0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,90.0,90.0,90.0,110.0,90.0,90.0,90.0,90.0,110.0,90.0,110,90.0,90.0,110.0,110.0,90.0,110.0,90.0
2,80.0,80.0,80.0,100.0,80.0,100.0,80.0,80.0,100.0,100.0,100,100.0,100.0,100.0,120.0,100.0,100.0,100.0
3,90.0,90.0,90.0,90.0,70.0,110.0,70.0,70.0,110.0,90.0,90,90.0,90.0,110.0,110.0,90.0,110.0,90.0
4,80.0,100.0,80.0,100.0,60.0,100.0,80.0,60.0,100.0,80.0,100,100.0,80.0,100.0,100.0,100.0,120.0,80.0
5,90.0,90.0,70.0,110.0,70.0,110.0,90.0,50.0,90.0,70.0,110,90.0,90.0,110.0,110.0,90.0,110.0,90.0
6,80.0,100.0,80.0,100.0,80.0,120.0,80.0,60.0,80.0,80.0,100,80.0,100.0,100.0,100.0,80.0,100.0,100.0
7,90.0,90.0,90.0,90.0,90.0,110.0,90.0,70.0,70.0,70.0,90,70.0,NaN,90.0,110.0,90.0,90.0,110.0
8,100.0,80.0,100.0,100.0,80.0,120.0,80.0,80.0,60.0,60.0,80,60.0,NaN,100.0,120.0,100.0,80.0,120.0
9,110.0,70.0,110.0,110.0,70.0,110.0,90.0,90.0,NaN,70.0,70,50.0,NaN,90.0,110.0,110.0,NaN,110.0


In [156]:
user_points_session_15_07 = users_points_history(session_15_07_track)
for k in user_points_session_15_07:
    if user_points_session_15_07[k][0] != 100:
        user_points_session_15_07[k].insert(0, 100)

In [157]:
# Dividindo rodadas do usuário 7 e 9
user_points_session_15_07[6] = user_points_session_15_07[7][17:]
user_points_session_15_07[8] = user_points_session_15_07[9][17:]

In [158]:
user_points_session_15_07 = dic_to_dataFrames(user_points_session_15_07)
user_points_session_15_07

,1,7,9,21,10,12,11,19,14,18,16,17,26,25,23,24,6,8
0,100.0,100,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,100.0,90,90.0,90.0,110.0,90.0,90.0,100.0,90.0,110.0,90.0,110.0,NaN,90.0,90.0,NaN,90.0,90.0
2,100.0,80,80.0,100.0,100.0,80.0,100.0,90.0,100.0,100.0,80.0,120.0,NaN,NaN,NaN,NaN,100.0,100.0
3,100.0,70,70.0,NaN,90.0,70.0,90.0,80.0,90.0,90.0,70.0,110.0,NaN,NaN,NaN,NaN,90.0,90.0
4,NaN,60,60.0,NaN,80.0,60.0,80.0,90.0,80.0,80.0,80.0,120.0,NaN,NaN,NaN,NaN,80.0,80.0
5,NaN,70,50.0,NaN,70.0,50.0,90.0,80.0,70.0,70.0,70.0,110.0,NaN,NaN,NaN,NaN,90.0,70.0
6,NaN,80,60.0,NaN,80.0,60.0,80.0,70.0,60.0,60.0,60.0,100.0,NaN,NaN,NaN,NaN,80.0,60.0
7,NaN,70,50.0,NaN,90.0,50.0,70.0,60.0,70.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN,90.0,50.0
8,NaN,60,60.0,NaN,80.0,40.0,60.0,70.0,80.0,60.0,60.0,NaN,NaN,NaN,NaN,NaN,100.0,40.0
9,NaN,50,70.0,NaN,70.0,50.0,50.0,80.0,70.0,70.0,50.0,NaN,NaN,NaN,NaN,NaN,110.0,NaN


In [159]:
user_points_session_15_07 = user_points_session_15_07[:][:17]

In [160]:
del user_points_session_15_07[1]
del user_points_session_15_07[21]
del user_points_session_15_07[26]
del user_points_session_15_07[25]
del user_points_session_15_07[23]
del user_points_session_15_07[24]

In [163]:
user_points_session_15_07

,7,9,10,12,11,19,14,18,16,17,6,8
0,100,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,90,90.0,110.0,90.0,90.0,100.0,90.0,110.0,90.0,110.0,90.0,90.0
2,80,80.0,100.0,80.0,100.0,90.0,100.0,100.0,80.0,120.0,100.0,100.0
3,70,70.0,90.0,70.0,90.0,80.0,90.0,90.0,70.0,110.0,90.0,90.0
4,60,60.0,80.0,60.0,80.0,90.0,80.0,80.0,80.0,120.0,80.0,80.0
5,70,50.0,70.0,50.0,90.0,80.0,70.0,70.0,70.0,110.0,90.0,70.0
6,80,60.0,80.0,60.0,80.0,70.0,60.0,60.0,60.0,100.0,80.0,60.0
7,70,50.0,90.0,50.0,70.0,60.0,70.0,50.0,50.0,NaN,90.0,50.0
8,60,60.0,80.0,40.0,60.0,70.0,80.0,60.0,60.0,NaN,100.0,40.0
9,50,70.0,70.0,50.0,50.0,80.0,70.0,70.0,50.0,NaN,110.0,NaN


### Contando acertos e erros pelos pontos

In [202]:
def count_hit_and_miss_from_points(df):
    dic = {}
    for user in df:
        case = 1
        dic[user] = {}
        for i in range(len(df[user])-1):
            current_points = df[user][i]
            next_points = df[user][i+1]
#             print(user, current_points)
#             if next_points >= 0:
            if next_points - current_points == 10:
                case_str = f"Caso{case}" 
                dic[user][case_str] = "acerto"
            elif next_points - current_points == -10:
                case_str = f"Caso{case}"
                dic[user][case_str] = "erro"
            else:
                case_str = f"Caso{case}"
                dic[user][case_str] = "sem resposta"
            case += 1
    return pd.DataFrame(dic)

In [212]:
table_hits_miss_11_07 = count_hit_and_miss_from_points(user_points_session_11_07)
table_hits_miss_11_07

,21,17,19,16,23,22,20,18,27,31,26,32,28,30,29,24,15,25
Caso1,erro,erro,erro,acerto,erro,erro,erro,erro,acerto,erro,acerto,erro,erro,acerto,acerto,erro,acerto,erro
Caso2,erro,erro,erro,erro,erro,acerto,erro,erro,erro,acerto,erro,acerto,acerto,erro,acerto,acerto,erro,acerto
Caso3,acerto,acerto,acerto,erro,erro,acerto,erro,erro,acerto,erro,erro,erro,erro,acerto,erro,erro,acerto,erro
Caso4,erro,acerto,erro,acerto,erro,erro,acerto,erro,erro,erro,acerto,acerto,erro,erro,erro,acerto,acerto,erro
Caso5,acerto,erro,erro,acerto,acerto,acerto,acerto,erro,erro,erro,acerto,erro,acerto,acerto,acerto,erro,erro,acerto
Caso6,erro,acerto,acerto,erro,acerto,acerto,erro,acerto,erro,acerto,erro,erro,acerto,erro,erro,erro,erro,acerto
Caso7,acerto,erro,acerto,erro,acerto,erro,acerto,acerto,erro,erro,erro,erro,sem resposta,erro,acerto,acerto,erro,acerto
Caso8,acerto,erro,acerto,acerto,erro,acerto,erro,acerto,erro,erro,erro,erro,sem resposta,acerto,acerto,acerto,erro,acerto
Caso9,acerto,erro,acerto,acerto,erro,erro,acerto,acerto,sem resposta,acerto,erro,erro,sem resposta,erro,erro,acerto,sem resposta,erro
Caso10,erro,erro,acerto,erro,acerto,acerto,acerto,acerto,sem resposta,erro,erro,acerto,sem resposta,erro,sem resposta,sem resposta,sem resposta,erro


In [222]:
count_user_h_m_11_07 = table_hits_miss_11_07.apply(pd.value_counts).T
count_user_h_m_11_07

,acerto,erro,sem resposta
21,9.0,7.0,NaN
17,6.0,10.0,NaN
19,12.0,4.0,NaN
16,7.0,9.0,NaN
23,8.0,8.0,NaN
22,11.0,4.0,1.0
20,6.0,5.0,5.0
18,8.0,8.0,NaN
27,2.0,6.0,8.0
31,6.0,9.0,1.0


In [210]:
table_hits_miss_15_07 = count_hit_and_miss_from_points(user_points_session_15_07)
table_hits_miss_15_07

,7,9,10,12,11,19,14,18,16,17,6,8
Caso1,erro,erro,acerto,erro,erro,sem resposta,erro,acerto,erro,acerto,erro,erro
Caso2,erro,erro,erro,erro,acerto,erro,acerto,erro,erro,acerto,acerto,acerto
Caso3,erro,erro,erro,erro,erro,erro,erro,erro,erro,erro,erro,erro
Caso4,erro,erro,erro,erro,erro,acerto,erro,erro,acerto,acerto,erro,erro
Caso5,acerto,erro,erro,erro,acerto,erro,erro,erro,erro,erro,acerto,erro
Caso6,acerto,acerto,acerto,acerto,erro,erro,erro,erro,erro,erro,erro,erro
Caso7,erro,erro,acerto,erro,erro,erro,acerto,erro,erro,sem resposta,acerto,erro
Caso8,erro,acerto,erro,erro,erro,acerto,acerto,acerto,acerto,sem resposta,acerto,erro
Caso9,erro,acerto,erro,acerto,erro,acerto,erro,acerto,erro,sem resposta,acerto,sem resposta
Caso10,acerto,erro,acerto,erro,acerto,erro,erro,acerto,erro,sem resposta,erro,sem resposta


In [223]:
count_user_h_m_15_07 = table_hits_miss_15_07.apply(pd.value_counts).T
count_user_h_m_15_07

,acerto,erro,sem resposta
7,4.0,12.0,NaN
9,4.0,12.0,NaN
10,5.0,11.0,NaN
12,2.0,14.0,NaN
11,7.0,9.0,NaN
19,5.0,10.0,1.0
14,6.0,10.0,NaN
18,5.0,10.0,1.0
16,5.0,11.0,NaN
17,3.0,3.0,10.0


## Salvando tabelas

In [233]:
table_hits_miss_11_07.to_excel("tragetoria_usuario_11_07.xlsx", index_label="Caso")
table_hits_miss_15_07.to_excel("tragetoria_usuario_15_07.xlsx", index_label="Caso")

In [232]:
table_count = pd.concat([count_user_h_m_11_07, count_user_h_m_15_07])
table_count.to_excel("acerto_erro_por_usuario.xlsx", index_label="usuário")